In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import json

train_path = '/content/drive/MyDrive/Colab Notebooks/Conversation_derailment/conv_derailment_data/data/cga/base/train.json'
test_path = '/content/drive/MyDrive/Colab Notebooks/Conversation_derailment/conv_derailment_data/data/cga/base/test.json'
val_path = '/content/drive/MyDrive/Colab Notebooks/Conversation_derailment/conv_derailment_data/data/cga/base/valid.json'

train_data = []
with open(train_path,'r') as f:
  for line in f:
    train_data.append(json.loads(line))

test_data = []
with open(test_path,'r') as f:
  for line in f:
    test_data.append(json.loads(line))

val_data = []
with open(val_path,'r') as f:
  for line in f:
    val_data.append(json.loads(line))

In [3]:
from collections import defaultdict

In [4]:
grouped_train = defaultdict(list)
for item in train_data:
    convo_id = item['convo_id']
    grouped_train[convo_id].append(item)


# duplicated convo_ids
duplicated_convo_ids = [k for k, v in grouped_train.items() if len(v) > 1]
print(duplicated_convo_ids)

[]


In [5]:
train_data[0]

{'src': ' i notice that earier that moved wiki _ link to bill chen citing wiki _ link, then you reverted this change, bill chen doesn\'t commonly go by william, his book is even penned as bill chen. from what i read in wp : commonname patrikr seems to be correct, examples given are names such as : * wiki _ link ( not wiki _ link ) * wiki _ link ( not wiki _ link ) i think this revert may have been a mistake unless you know otherwise? ▪ [UNK] ▪ [SEP]  chen was known in the poker world as " william " for years before he became commonly known as " bill ". i changed it back because incidences online including usenet are roughly equal, nothing at all like bill clinton and william clinton, and in equal cases using the real name seems the best choice. ( the external _ link page is especially pschizo... willam in the page title, bill in the page text ). however i suppose the book is the trump card, so using the name on the book is probably best.',
 'reply': [101,
  1045,
  2156,
  2054,
  2017

In [6]:
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def data_for_gpt2(data):
  src_text = data['src']
  reply_tokens = data['reply']
  tgt = data['tgt']
  decoded_reply = tokenizer_bert.decode(reply_tokens, skip_special_tokens=True)
  combined_text = f"<SRC>: {src_text}  <REPLY>: {decoded_reply} <|endofcontext|>"
  return combined_text, tgt

In [8]:
train_data_gpt2 = []
tgt_train = []
for item in train_data:
  formatted_text, tgt = data_for_gpt2(item)
  train_data_gpt2.append(formatted_text)
  tgt_train.append(tgt)

In [9]:
train_data_gpt2[0]

'<SRC>:  i notice that earier that moved wiki _ link to bill chen citing wiki _ link, then you reverted this change, bill chen doesn\'t commonly go by william, his book is even penned as bill chen. from what i read in wp : commonname patrikr seems to be correct, examples given are names such as : * wiki _ link ( not wiki _ link ) * wiki _ link ( not wiki _ link ) i think this revert may have been a mistake unless you know otherwise? ▪ [UNK] ▪ [SEP]  chen was known in the poker world as " william " for years before he became commonly known as " bill ". i changed it back because incidences online including usenet are roughly equal, nothing at all like bill clinton and william clinton, and in equal cases using the real name seems the best choice. ( the external _ link page is especially pschizo... willam in the page title, bill in the page text ). however i suppose the book is the trump card, so using the name on the book is probably best.  <REPLY>: i see what you saying i just read his p

In [10]:
tgt_train[0]

False

In [11]:
val_data_gpt2 = []
tgt_val = []
for item in val_data:
  formatted_text, tgt = data_for_gpt2(item)
  val_data_gpt2.append(formatted_text)
  tgt_val.append(tgt)

In [12]:
val_data_gpt2[0]

'<SRC>:  you have been blocked for three hours for incivility. we cannot tolerate these types of edits : [SEP]  on the contrary, the edits you should not tolerate are the other side preaching their incorrect disambiguation. - - wiki _ link )  <REPLY>: i have unblocked you to allow you to reply to your rfar. <|endofcontext|>'

In [13]:
tgt_val[0]

False

In [14]:
test_data_gpt2 = []
tgt_test = []
for item in val_data:
  formatted_text, tgt = data_for_gpt2(item)
  test_data_gpt2.append(formatted_text)
  tgt_test.append(tgt)

In [15]:
from transformers import AutoTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

In [16]:
from transformers import GPT2ForSequenceClassification, GPT2Config
import torch
import torch.nn.functional as F

class GPT2ForSequenceClassificationMeanPooling(GPT2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        self.gpt2 = self.transformer  # Assuming the GPT2ForSequenceClassification model has a 'transformer' attribute
        self.score = torch.nn.Linear(config.hidden_size, config.num_labels)

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        labels=None,
        **kwargs
    ):
        outputs = self.gpt2(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **kwargs,
        )

        # Get the hidden states from the last layer
        hidden_states = outputs.last_hidden_state

        # Apply mean pooling on the hidden states
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
        sum_hidden_states = torch.sum(hidden_states * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        mean_hidden_states = sum_hidden_states / sum_mask

        # Pass the pooled hidden states through the classification head
        logits = self.score(mean_hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    loss_fct = torch.nn.MSELoss()
                    loss = loss_fct(logits.view(-1), labels.view(-1))
                else:
                    loss_fct = torch.nn.CrossEntropyLoss()
                    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            else:
                raise ValueError("You should supply an instance of `PreTrainedModel` or a `config`")

        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output


In [17]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [18]:
### Loading Model and Tokenizers
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassificationMeanPooling.from_pretrained('gpt2', num_labels=2)
print(model)
model.to(device)

Some weights of GPT2ForSequenceClassificationMeanPooling were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassificationMeanPooling(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=True)
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop)

GPT2ForSequenceClassificationMeanPooling(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=True)
  (gpt2): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop)

In [19]:
## Defining Special Tokens

special_tokens = {'additional_special_tokens': ['<SRC>:', ' <REPLY>:', ' <|endofcontext|>','[UNK]','[SEP]']}
num_added = tokenizer.add_special_tokens(special_tokens)
print(num_added)
model.resize_token_embeddings(len(tokenizer))

5


Embedding(50262, 768)

In [20]:
# Tokenizing Data
tokenizer.pad_token = tokenizer.eos_token
train_encodings = tokenizer(train_data_gpt2, truncation=False, padding=False)


Token indices sequence length is longer than the specified maximum sequence length for this model (1110 > 1024). Running this sequence through the model will result in indexing errors


In [21]:
max_length = max(len(x) for x in train_encodings.input_ids)


In [22]:
max_length

1110

In [23]:
val_encodings = tokenizer(val_data_gpt2, truncation=False, padding=False)

In [24]:
val_max_length = max(len(x) for x in val_encodings.input_ids)
val_max_length

1072

In [25]:
test_encodings = tokenizer(test_data_gpt2, truncation=False, padding=False)

In [26]:
test_max_length = max(len(x) for x in test_encodings.input_ids)
test_max_length

1072

In [27]:
## Redefining Tokenizer with padding and max length
tokenizer.pad_token = tokenizer.eos_token
train_inputs = tokenizer(train_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')

# Create torch dataset
input_ids = train_inputs['input_ids']
attention_mask = train_inputs['attention_mask']
labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_train])
train_dataset = TensorDataset(input_ids, attention_mask, labels)

In [28]:
len(input_ids[0])

1024

In [29]:
val_inputs = tokenizer(val_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')
val_input_ids = val_inputs['input_ids']
val_attention_mask = val_inputs['attention_mask']
val_labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_val])
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)

In [30]:
test_inputs = tokenizer(test_data_gpt2, truncation=True, padding=True, max_length=1024, return_tensors='pt')
test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']
test_labels = torch.tensor([0 if tgt == False else 1 for tgt in tgt_test])
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

In [31]:
## CREATING DATALOADERS
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [32]:
from torch.optim import AdamW

In [33]:
optimizer = AdamW(model.parameters(), lr=1e-5)

In [34]:
import copy

In [35]:
from sklearn.metrics import f1_score

#Training and Validation Loop
model.train()
num_epochs = 1
train_losses = []
val_losses = []
f1_scores = []
best_val_loss = float('inf')


for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} of {num_epochs}")
    train_loss = 0

    for batch in tqdm(train_dataloader, desc='Training Progress', miniters=10):
        input_ids, attention_mask, batch_labels = batch
        input_ids, attention_mask, batch_labels = input_ids.to(device), attention_mask.to(device), batch_labels.to(device)
        optimizer.zero_grad()
        model.config.pad_token_id = tokenizer.eos_token_id
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs[0]
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_dataloader)
    train_losses.append(avg_train_loss)
    print(f"Training loss: {avg_train_loss:.2f}")

    val_loss = 0
    model.eval()
    val_preds = []
    val_true_labels = []
    with torch.no_grad():
      for batch in tqdm(val_dataloader, desc='Validation Progress', miniters = 10):
        input_ids, attention_mask, batch_labels = batch
        input_ids, attention_mask, batch_labels = input_ids.to(device), attention_mask.to(device), batch_labels.to(device)
        model.config.pad_token_id = tokenizer.eos_token_id
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=batch_labels)
        loss = outputs.loss
        val_loss += loss.item()

        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_true_labels.extend(batch_labels.cpu().numpy())

      avg_val_loss = val_loss / len(val_dataloader)
      val_losses.append(avg_val_loss)
      f1 = f1_score(val_true_labels, val_preds, average='macro')
      f1_scores.append(f1)
      print(f"Validation loss: {avg_val_loss:.2f}, F1 score: {f1:.2f}")
      # print(f"Confusion Matrix: {confusion_matrix(val_true_labels, val_preds)}")
      # print(f"Classification Report: {classification_report(val_true_labels, val_preds)}")
      model.train()

Epoch 1 of 1


Training Progress:   0%|          | 1/1254 [01:20<27:56:20, 80.27s/it]


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(f"Confusion Matrix:\n {confusion_matrix(val_true_labels, val_preds)}")
print(f"Classification Report:\n {classification_report(val_true_labels, val_preds)}")